In [1]:
import pandas as pd
import numpy as np
import sklearn

In [2]:
falencia_teste = pd.read_csv('falencia-teste.csv')
falencia_trinamento = pd.read_csv('falencia-treinamento.csv')
modelo_submissao = pd.read_csv('modelo-submissao.csv')

In [3]:
falencia_teste

,;I1;I2;I3;I4;I5;I6;I7;I8;I9;I10;M1;M2;M3;M4;M5;M6;M7;M8;M9;M10
0,0;2.89;3.56;4.31;5.26;2.86;5.26;2.38;3.04;5.84...
1,1;7.08;5.88;2.81;6.71;4.02;6.71;6.85;3.47;6.96...
2,2;4.09;3.54;3.66;4.85;6.09;4.85;4.81;3.33;6.81...
3,3;9.36;3.32;5.93;4.94;3.01;4.94;7.08;4.68;3.62...
4,4;3.38;6.48;3.02;6.02;4.87;6.02;5.70;4.48;6.70...
...,...
95,95;1.23;5.76;5.96;2.39;4.04;2.39;1.76;7.16;3.7...
96,96;5.29;3.90;5.25;5.35;3.33;5.35;4.68;4.47;4.3...
97,97;3.56;5.74;3.33;5.93;7.00;5.93;5.02;4.89;6.3...
98,98;4.64;3.70;4.33;5.00;7.68;5.00;6.13;4.66;5.6...
